In [101]:
import time
import pandas as pd
from wikiapi import WikiApi
wiki = WikiApi()
import re
from gensim.models import Word2Vec
#import word2vec
from scipy.spatial.distance import cosine
import numpy as np
import heapq
import string


In [102]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [134]:


def remove_punctuation(s):
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in s if ch not in exclude)
    return s
#TODO: fix this or dont use
def get_longword(s, n):
    s = re.split(' ', s)
    return heapq.nlargest(n, s, key=len)

#TODO: make this work with one word/period
def get_avg_vec(words):
	try:
		words = re.split(' ', words)
	except:
		pass
	nwords = 0
	featureVec = np.zeros((num_features,))#,dtype="float32")
	for word in words:
		if word in index2word_set:
			nwords = nwords + 1.
			featureVec = np.add(featureVec,model[word])
	if nwords > 0:
		featureVec = np.divide(featureVec,nwords)
	return featureVec

def get_tvec(row):
    if row['correctAnswer'] == 'A':
        a = row['aavec']
        return a
    elif row['correctAnswer'] == 'B':
        a = row['bavec']
        return a
    elif row['correctAnswer'] == 'C':
        a = row['cavec']
        return a
    elif row['correctAnswer'] == 'D':
        a = row['davec']
        return a
    
def run_through_theta(v):
    return v.dot(theta.as_matrix())

def get_winner_from_avecs(row):
	try:	
		dists = []
		for col in row['answerA':'answerD']:
		#TODO: if 'not' in words: 1-avec
			avec = get_avg_vec(col)
			#if 'not' in re.split(' ',col):
			#	avec = 1-avec
			#	print('NOT')
			#why nan errors?
			if sum(np.isfinite(avec)) < 300:
				print('avec')
				print(avec)
			if  sum(np.isfinite(row['pvec'])) < 300:
				print('pvec')
				print(row['pvec'])
			dist = cosine(avec, row['pvec'])
			dists.append(dist)
		m = np.nanmin(dists)
		best = [i for i, j in enumerate(dists) if j == m]
		if best == [0]:
			return 'A'
		if best == [1]:

			return 'B'
		if best == [2]:
			return 'C'
		if best == [3]:
			return 'D'
		else:
			print('returning C becuase no best cosine')
			print(len(best))
			print(dists)
			print(m)
			return 'C'
	except:
		print('some error, returning C')
		return 'C'

In [3]:
start = time.time()
model = Word2Vec.load_word2vec_format('/Users/liamconnell/Downloads/GoogleNews-vectors-negative300.bin', binary = True)
lap1 = time.time()
print('data gathered: %s' % (lap1 - start))

data gathered: 103.47626209259033


In [1]:
model['queen']

NameError: name 'model' is not defined

In [4]:
index2word_set = set(model.index2word)
num_features = 300

In [78]:
data  = pd.read_csv('../input/validation_set.tsv', '\t')
#theta = pd.read_csv('../input/theta_100-a_e-10.csv')

In [156]:
data  = pd.read_csv('../input/validation_set_mod-5.csv')


In [157]:
data.head()

,Unnamed: 0,id,question,answerA,answerB,answerC,answerD,keyword,words
0,0,102501,A meter is industry by developing a process of...,cool foods during shipping,yard,nervous system,birds of prey,"['developing', 'industry', 'examples', 'proces...",['Developing\n\nDeveloping is a 1994 short fil...
1,1,102502,Which part of directly involved in recycling t...,panting,urethra,predators,water,"['reproduction,', 'temperature', '__________.'...","['Reproduction, Fertility and Development\n\nR..."
2,2,102503,Which of the seeds grow better in the light or...,muscular system,keep one group in a light place and another in...,skates and rays,nervous tissue,"['framework', 'provides', 'organs?', 'better',...",['Framework\n\nFramework may refer to:\n\nComp...
3,3,102504,Which of the hermaphroditic. This growing seas...,coniferous forest,no reproductive organs,there is less water loss through small leaves,perennial,"['hermaphroditic.', 'growing', 'season', 'Whic...","['hermaphroditic.', 'Growing\n\nGrowing may re..."
4,4,102505,When marsupials following is not a transport o...,heart.,glucagon.,hatched.,to protect vital organs.,"['marsupials', 'following', 'transport', 'resp...",['Marsupial\n\nMarsupials are an infraclass of...


In [150]:
theta = pd.read_csv('../input/qkeywd-5_theta_1000-a_e-100.csv')

In [129]:
theta = pd.DataFrame(np.identity(300))

In [114]:
data.question = data.question.apply(remove_punctuation)
data.answerA = data.answerA.apply(remove_punctuation)
data.answerB = data.answerB.apply(remove_punctuation)
data.answerC = data.answerC.apply(remove_punctuation)
data.answerD = data.answerD.apply(remove_punctuation)

In [145]:
data['keyword'] =data['question'].apply(get_longword, args = (5,))

#I get to choose this
data['qvec'] = data.keyword.apply(get_avg_vec) 


In [138]:
pd.Series.apply?

In [146]:
#data['qvec'] = data.question.apply(get_avg_vec) 
data['akeyword'] =data['answerA'].apply(get_longword, args = (1,))
data['bkeyword'] =data['answerB'].apply(get_longword, args = (1,))
data['ckeyword'] =data['answerC'].apply(get_longword, args = (1,))
data['dkeyword'] =data['answerD'].apply(get_longword, args = (1,))

data['aavec'] = data['akeyword'].apply(get_avg_vec)
data['bavec'] = data['bkeyword'].apply(get_avg_vec)
data['cavec'] = data['ckeyword'].apply(get_avg_vec)
data['davec'] = data['dkeyword'].apply(get_avg_vec)

In [151]:
data['aavec'] = data['answerA'].apply(get_avg_vec)
data['bavec'] = data['answerB'].apply(get_avg_vec)
data['cavec'] = data['answerC'].apply(get_avg_vec)
data['davec'] = data['answerD'].apply(get_avg_vec)

In [152]:
data['pvec'] = data.qvec.apply(run_through_theta)

In [153]:
data['closest_avec'] = data.apply(get_winner_from_avecs, axis=1)


returning C becuase no best cosine
2
[0.91083864667413383, 0.86769536778558765, 0.85237956112431912, 0.85237956112431912]
0.852379561124
returning C becuase no best cosine
4
[0.66321268371433473, 0.66321268371433473, 0.66321268371433473, 0.66321268371433473]
0.663212683714
returning C becuase no best cosine
4
[0.98343039705456836, 0.98343039705456836, 0.98343039705456836, 0.98343039705456836]
0.983430397055
returning C becuase no best cosine
2
[0.65485057611822106, 0.69121066223768102, 0.65557276765948469, 0.65485057611822106]
0.654850576118
returning C becuase no best cosine
2
[0.61075096046385546, 0.61075096046385546, 0.59567575426978692, 0.59567575426978692]
0.59567575427
returning C becuase no best cosine
2
[0.80971383126343266, 0.80971383126343266, 0.86611296077017941, nan]
0.809713831263
returning C becuase no best cosine
2
[0.89175419890918661, 0.91414324162014271, 0.91048146100569838, 0.89175419890918661]
0.891754198909
returning C becuase no best cosine
4
[0.90952176319195122,

In [155]:
sample = pd.read_csv('../input/sample_submission.csv')
sample['correctAnswer'] = data['closest_avec']
sample.to_csv('../output/qkeywd-5_theta-1000_a-e-100.csv', index=False)